In [1]:
import numpy as np
import pandas as pd

from pandas import DataFrame, Series

def get_returns(file_path):
    data_frame = pd.read_excel(file_path, index_col='Date', parse_dates=['Date'], squeeze=False)
    returns = data_frame['Change %'].str.rstrip(' %').astype('float')
    returns.dropna(inplace=True)
    returns.name = 'Total Returns'
    return returns

def get_cumulative_returns(returns):
    return (((returns / 100) + 1).cumprod() - 1) * 100

def get_returns_extrema(returns, n):
    sorted = returns.sort_values()
    top = sorted.head(n=n)
    bottom = sorted.tail(n=n)
    return pd.concat([top, bottom])

def get_rolling_calculations(returns, callback, rolling_period=30, mult_factor=1, is_series=True):
    calcs = {}
    count = returns.count() if is_series else returns.count()['Total Returns Index']
    last_index = count - rolling_period
    for index in range(last_index):
        date = returns.index[index+rolling_period]
        calc = callback(returns.iloc[index:index+rolling_period])*mult_factor
        calcs[date] = calc
    return Series(calcs)

In [2]:
# ETL
stock_file_path = 'mdt_historical_quotes.xlsx'
index_file_path = 'spy_historical_quotes.xlsx'

stock_returns = get_returns(stock_file_path)
index_returns = get_returns(index_file_path)
returns = pd.merge(index_returns, stock_returns,on='Date', suffixes=(' Index', ' Stock'))

In [3]:
# Series Analysis
count_returns_extrema = 5
annualize_std_factor = 252**.5

stock_cumulative_returns = get_cumulative_returns(stock_returns)
stock_returns_extrema = get_returns_extrema(stock_returns,n=count_returns_extrema)
stock_rolling_stds = get_rolling_calculations(stock_returns,Series.std,mult_factor=annualize_std_factor)

index_cumulative_returns = get_cumulative_returns(index_returns)
index_returns_extrema = get_returns_extrema(index_returns,n=count_returns_extrema)
index_rolling_stds = get_rolling_calculations(index_returns,Series.std,mult_factor=annualize_std_factor)

In [4]:
# DataFrame Analysis
callback = lambda returns: DataFrame.corr(returns).iloc[1,0]
rolling_corrs = get_rolling_calculations(returns,callback,is_series=False)

2017-02-16    0.040458
2017-02-17    0.113991
2017-02-21    0.087086
2017-02-22    0.143566
2017-02-23    0.133162
                ...   
2021-12-22    0.398741
2021-12-23    0.398795
2021-12-27    0.423278
2021-12-28    0.453587
2021-12-29    0.452055
Length: 1220, dtype: float64